In [ ]:
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
from mfml.resources.data import load_mnist
MNIST = load_mnist()
images = MNIST['data'].astype(np.double)
labels = MNIST['target'].astype(np.int64)

In [ ]:
plt.figure(figsize=(4,4))
plt.imshow(images[0].reshape(28,28), cmap='gray');
plt.grid(False)

## PCA

Now we will implement PCA. Before we do that, let's pause for a moment and
think about the steps for performing PCA. Assume that we are performing PCA on
some dataset $\boldsymbol X$ for $M$ principal components. 
We then need to perform the following steps, which we break into parts:

1. Data normalization (`normalize`).
2. Find eigenvalues and corresponding eigenvectors for the covariance matrix $S$.
   Sort by the largest eigenvalues and the corresponding eigenvectors (`eig`).
3. Compute the orthogonal projection matrix and use that to project the data onto the subspace spanned by the eigenvectors.

### Data normalization `normalize`


In [ ]:
def normalize(X):
    """Normalize the given dataset X to have zero mean & 1 unit of standard deviation.
    Args:
        X: ndarray, dataset of shape (N,D)
    
    Returns:
        (Xbar, mean): tuple of ndarray, Xbar is the normalized dataset
        with mean 0 & standard deviation of 1.
    """
    mu = X.mean(axis=0)
    sd = X.std(axis=0)
    # it can occur that there is NAN's when computing stdev, impute 1's
    sd_ed = sd.copy()
    sd_ed[sd == 0] = 1.
    x_norm = (X - mu) / sd_ed
    return x_norm, mu, sd_ed

### Compute eigenvalues and eigenvectors `eig`

In [ ]:
def eig(S):
    """Compute the eigenvalues and corresponding eigenvectors
        for the covariance matrix S.
    Args:
        S: ndarray, covariance matrix

    Returns:
        (eigvals, eigvecs): ndarray, the eigenvalues and eigenvectors

    Note:
        the eigenvals and eigenvecs should be sorted in descending
        order of the eigen values
    """
    eigvals, eigvecs = np.linalg.eig(S)
    sort_ind = np.argsort(eigvals)[::-1]
      
    return eigvals[sort_ind], eigvecs[:, sort_ind]

Next given a orthonormal basis spanned by the eigenvectors, we will compute the projection matrix.

In [ ]:
def projection_matrix(B):
    """Compute the projection matrix onto the space spanned by `B`
    Args:
        B: ndarray of dimension (D, M), the basis for the subspace
    
    Returns:
        P: the projection matrix
    """
    P = (B@(np.linalg.inv(B.T@B))@B.T)
    return P

### PCA

In [ ]:
def PCA(X, num_components, recon_origin=True):
    """
    Args:
        X: ndarray of size (N, D), where D is the dimension of the data,
           and N is the number of datapoints
        num_components: the number of principal components to use.
        recon_origin: whether to return the reconstruction matrix normalised (True) or not
    Returns:
        the reconstructed data, the sample mean of the X, principal values
        and principal components
    """
    X_norm, mean, sd = normalize(X)
    S = np.cov(X_norm, rowvar=False, bias=True)
    # find eigenvalues and corresponding eigenvectors for S
    eig_vals, eig_vecs = eig(S)
    # take the top `num_components` of eig_vals and eig_vecs,
    # this will be the corresponding principal values and components
    principal_vals, principal_components = eig_vals[:num_components], eig_vecs[:, :num_components]
    P = projection_matrix(principal_components)
    if recon_origin:
        reconst = (P @ X_norm.T).T * sd + mean
    else:
        reconst = (P @ X_norm.T).T
    # reconstruct the data from the using the basis spanned by the principal components
    # Notice that we have subtracted the mean from X and divided by the stdev, so make sure 
    # that you add it back to the reconstructed data
    return reconst, mean, principal_vals, principal_components

## Visualize PCA

In [ ]:
def draw_vector(v0, v1, ax=None, label=None):
    """Draw a vector from v0 to v1."""
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0, 
                    color='k')
    ax.annotate('', v1, v0, arrowprops=arrowprops, label=label)

# visualize what PCA does on a 2D toy dataset.
mvn = scipy.stats.multivariate_normal(
    mean=np.ones(2), 
    cov=np.array([[1, 0.8], [0.8, 1]])
)

X = mvn.rvs((100,), random_state=np.random.RandomState(0))

num_components = 1
X_reconst, mean, principal_values, principal_components = PCA(X, num_components)

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], label='data')
for (princial_variance, principal_component) in (zip(principal_values, principal_components.T)):
    draw_vector(
        mean, mean + np.sqrt(princial_variance) * principal_component, 
        ax=ax)
ax.scatter(X_reconst[:, 0], X_reconst[:, 1], label='reconstructed')
plt.axis('equal');
plt.legend();
ax.set(xlabel='$\mathbf{x}_0$', ylabel='$\mathbf{x}_1$');

## PCA for MNIST digits

In [ ]:
# pre-processing
NUM_DATAPOINTS = 1000
X = (images.reshape(-1, 28 * 28)[:NUM_DATAPOINTS]) / 255.
Xbar, mu, std = normalize(X)

> How many principal components do we need
> in order to reach a Mean Squared Error (MSE) of less than 100 for our dataset?

In [ ]:
def mse(predict, actual):
    return np.square(predict - actual).sum(axis=1).mean()

In [ ]:
loss = []
reconstructions = []
for num_component in range(1, 100, 5):
    reconst, _, _, _ = PCA(Xbar, num_component, recon_origin=False)
    error = mse(reconst, Xbar)
    reconstructions.append(reconst)
    print('n = {:d}, reconstruction_error = {:f}'.format(num_component, error))
    loss.append((num_component, error))

reconstructions = np.asarray(reconstructions)
reconstructions = reconstructions * std + mu # bring back to original space of the reconstructed image
loss = np.asarray(loss)

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss[:,0], loss[:,1]);
ax.axhline(100, linestyle='--', color='r', linewidth=2)
ax.set(xlabel='num_components', ylabel='MSE', title='MSE vs number of principal components');

Numbers don't tell us much. Just what does it mean qualitatively for the loss to decrease from around 450 to less than 100? In the next cell, we draw the original eight as the leftmost image. Then we show the reconstruction of the image on the right, in descending number of principal components used.

In [ ]:
@interact(image_idx=(0, 1000))
def show_num_components_reconst(image_idx):
    fig, ax = plt.subplots(figsize=(20., 20.))
    actual = X[image_idx]
    x = np.concatenate([actual[np.newaxis, :], reconstructions[:, image_idx]])
    ax.imshow(np.hstack(x.reshape(-1, 28, 28)[np.arange(10)]),
              cmap='gray');
    ax.axvline(28, color='orange', linewidth=2)